## Урок 2. Парсинг html, BeautifulSoup

### Задание 1.

Необходимо собрать информацию о вакансиях на вводимую должность 
(используем input или через аргументы) с сайта superjob.ru и hh.ru. 
Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). 
Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. 
Данная структура должна быть одинаковая для вакансий с обоих сайтов. 
Общий результат можно вывести с помощью dataFrame через pandas.

In [185]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
from requests import get
import string

headers   = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
hh = 0
sj = 1

# список профессий для поиска
prof_list = [['Data+scientist','Слесарь','Главный+бухгалтер','Библиотекарь'],   
             ['data-scientist','slesar','buhgalter-glavnyj','bibliotekar']]     

# список страниц для поиска
page_list = [['0', '1'],
             ['0', '1']]     

# список сайтов для поиска
site_list = ['https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=Prof_Name&page=Page_Number',
             'https://russia.superjob.ru/vakansii/Prof_Name.html?page=Page_Number'] 

# список валют
currency_list = ['руб.', 'USD', 'EUR', 'гривен']

# возвращает значение максимального и минимального размера оплаты из объявления
def find_salary(vacancy_salary):
    
    # удаляем побелы и специальные символы
    vacancy_salary = vacancy_salary.replace(u'\xa0', '')
    vacancy_salary = vacancy_salary.replace(' ', '')

    # удаляем валюту из строки и сохраняем ее наименование в переменной salary_currency
    for currency in currency_list:
        if vacancy_salary.rfind(currency)>0:
            salary_currency = currency
            vacancy_salary = vacancy_salary.replace(currency, '')
    
    # ищем минимальную и максимальную зарплату в строке   
    if vacancy_salary == "":      # значения по умолчанию, если в вакансии нет указания з\п
        salary_currency = "руб."  
        vacancy_salary_min = 0
        vacancy_salary_max = 99999999
    elif vacancy_salary.startswith('от'):
        vacancy_salary = vacancy_salary.replace('от', '')
        vacancy_salary_min = int(vacancy_salary)
        vacancy_salary_max = 99999999
    elif vacancy_salary.startswith('до'):
        vacancy_salary = vacancy_salary.replace('до', '')
        vacancy_salary_min = 0
        vacancy_salary_max = int(vacancy_salary)
    elif vacancy_salary.isdigit():
        vacancy_salary_min = int(vacancy_salary)
        vacancy_salary_max = int(vacancy_salary)
    else:
        vacancy_salary = vacancy_salary.replace('—', '-')
        vacancy_salary = vacancy_salary.split('-',1)
        vacancy_salary_min = int(vacancy_salary[0])
        vacancy_salary_max = int(vacancy_salary[1])
 
    return vacancy_salary_min, vacancy_salary_max, salary_currency


# возвращает перечень структурированных данных из переданного тега вакансии c заданного сайта
def find_vacancy(vacancy, site):
    
    # поиск и разбо информации в теге
    if site == hh:
        vacancy_name   = vacancy.find_all('div',{'class':'resume-search-item__name'})[site].getText()
        vacancy_href   = str(vacancy.find_all('a',  {'class':'bloko-link'})[site]).split()[7][5:].strip('"') 
        vacancy_site   = "hh.ru"
        vacancy_salary = vacancy.find_all('div',{'class':'vacancy-serp-item__sidebar'})[site].getText()      
    elif site == sj:
        vacancy_name   = vacancy.find('a',{'class':'icMQ_'}).getText()
        vacancy_href   = "www.superjob.ru"+str(vacancy.find('a',{'class':'icMQ_'})).split('href="')[1].split('"')[0]
        vacancy_site   = "superjob.ru"
        vacancy_salary = vacancy.find('span',{'class':'f-test-text-company-item-salary'}).getText()    
        # перевод в формат hh - пустая строка, если з\п не установлена
        if vacancy_salary == 'По договорённости':
            vacancy_salary = ""  
        # перевод в формат hh - так как replace не работает с данными символами заменяем ₽ на руб.
        currency_pos = vacancy_salary.find('₽')
        if currency_pos > 0:
            vacancy_salary = vacancy_salary[:currency_pos]+'руб.'
    else:
        vacancy_name   = "неизвестная вакансия"
        vacancy_href   = "неизвестная ссылка" 
        vacancy_site   = "неизвестный сайт"
        vacancy_salary = ""         
        
    vacancy_salary_min, vacancy_salary_max, salary_currency = find_salary(vacancy_salary)
    
    vacancy_item = {}
    vacancy_item['name'] = vacancy_name
    vacancy_item['href'] = vacancy_href
    vacancy_item['site'] = vacancy_site
    vacancy_item['salary_min'] = vacancy_salary_min
    vacancy_item['salary_max'] = vacancy_salary_max
    vacancy_item['salary_currency'] = salary_currency

    return vacancy_item

# определяем пустой список с вакансиями
vacancies = []

# поиск на HeadHunter
for prof_name in prof_list[hh]:
    for page_number in page_list[hh]:
        site_name = site_list[hh].replace('Prof_Name',prof_name).replace('Page_Number',page_number)
        link = get(site_name, headers=headers)
        if link.status_code == 200:
            html = bs(link.text,'html')
            vacancy_list = html.find_all('div',{'class':'vacancy-serp-item__row_header'})
            # выбор и парсинг необходимых данных из списка ссылок
            for vacancy_message in vacancy_list:
                vacancy_item = find_vacancy(vacancy_message, hh)
                vacancies.append(vacancy_item)
    
# поиск на SuperJob
for prof_name in prof_list[sj]:
    for page_number in page_list[sj]:
        site_name = site_list[sj].replace('Prof_Name',prof_name).replace('Page_Number',page_number)
        link = get(site_name, headers=headers)
        if link.status_code == 200:
            html = bs(link.text,'html')
            vacancy_list = html.find_all('div',{'class':'f-test-vacancy-item'})
            # выбор и парсинг необходимых данных из списка ссылок
            for vacancy_message in vacancy_list:
                vacancy_item = find_vacancy(vacancy_message, sj)
                vacancies.append(vacancy_item)
    
for vacancy_item in vacancies:
    print(vacancy_item)


{'name': 'Big Data Engineer', 'href': 'https://hh.ru/vacancy/35787520?query=Data%20scientist', 'site': 'hh.ru', 'salary_min': 250000, 'salary_max': 99999999, 'salary_currency': 'руб.'}
{'name': 'Junior ML Engineer / Data scientist', 'href': 'https://hh.ru/vacancy/35459810?query=Data%20scientist', 'site': 'hh.ru', 'salary_min': 80000, 'salary_max': 120000, 'salary_currency': 'руб.'}
{'name': 'Middle/Senior data scientist (fraud prevention)', 'href': 'https://hh.ru/vacancy/35467008?query=Data%20scientist', 'site': 'hh.ru', 'salary_min': 140000, 'salary_max': 220000, 'salary_currency': 'руб.'}
{'name': 'Data scientist', 'href': 'https://hh.ru/vacancy/35546587?query=Data%20scientist', 'site': 'hh.ru', 'salary_min': 130000, 'salary_max': 500000, 'salary_currency': 'руб.'}
{'name': 'Junior (Middle) Data Scientist / Специалист по машинному обучению', 'href': 'https://hh.ru/vacancy/35514256?query=Data%20scientist', 'site': 'hh.ru', 'salary_min': 0, 'salary_max': 99999999, 'salary_currency': 'р